In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
import tensorflow_hub as hub

from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers
from keras.applications.vgg16 import VGG16

: 

: 

The ImageDataGenerator generates image tensors with real time data augmentation.
The adam optimiser is a stochastic gradient descent algorithm, default learning rate is 0.001.

In [9]:
validation_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
valid_gen = validation_datagen
train_gen = validation_datagen

rescale=1./255 converts all pixels from a range of [0, 255] to [0, 1]

In [10]:
IMAGE_SIZE = (255, 255)
BATCH_SIZE = 32

In [11]:
data_dir = "../input/plantvillage-dataset/color"
train_generator = train_gen.flow_from_directory(
    data_dir,
    target_size = IMAGE_SIZE,
    batch_size = BATCH_SIZE,
    subset= "training",
    class_mode = "categorical",
    color_mode = "rgb"
)

valid_generator = valid_gen.flow_from_directory(
    data_dir,
    target_size = IMAGE_SIZE,
    batch_size = BATCH_SIZE,
    subset= "validation",
    class_mode = "categorical",
    color_mode = "rgb"  
)

In [12]:
model = VGG16(include_top=False, input_shape=(255,255,3), weights="imagenet")
for layer in model.layers:
    layer.trainable = False
x = Flatten()(model.output)
dense1 = Dense(1024, activation="relu")(x)
prediction = Dense(38, activation="softmax")(dense1)
model = Model(inputs=model.inputs, outputs=prediction)

In [13]:
model.compile(
    optimizer = 'adam',
    loss = tf.keras.losses.CategoricalCrossentropy(),
    metrics  = ['accuracy']
    )

In [ ]:
epochs = 10

In [14]:
history = model.fit_generator(
        train_generator,
        steps_per_epoch = train_generator.n // train_generator.batch_size,    #The 2 slashes division return rounded integer
        epochs = epochs,
        validation_data = valid_generator,
        validation_steps = valid_generator.n // valid_generator.batch_size
        )

In [42]:
classnames = valid_generator.class_indices 
classes = []
for classname in classnames.keys():
    classes.append(classname)
print(classes)

In [44]:
# Import OpenCV
import cv2
# Utility
import itertools
import random
from collections import Counter
from glob import iglob
import os
import numpy as np
import matplotlib.pylab as plt

def load_image(filename):
    img = cv2.imread(os.path.join(data_dir, filename))
    img = cv2.resize(img, (IMAGE_SIZE[0], IMAGE_SIZE[1]) )
    img = img /255
    return img

def predict(image):
    probabilities = model.predict(np.asarray([image]))[0]
    class_idx = np.argmax(probabilities)
    return {classes[class_idx]: probabilities[class_idx]}

for idx, filename in enumerate(random.sample(valid_generator.filenames, 5)):
    print("SOURCE: class: %s, file: %s" % (os.path.split(filename)[0], filename))
    img = load_image(filename)
    prediction = predict(img)
    print("PREDICTED: class: %s, confidence: %f" % (list(prediction.keys())[0], list(prediction.values())[0]))
    plt.imshow(img)
    plt.figure(idx)    
    plt.show()